In [10]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType,TimestampType)
from pyspark.sql.functions import *
import pandas as pd


In [11]:
spark = SparkSession.builder.appName('Custmer data').getOrCreate()

In [12]:
df_customer= spark.read.json("cdw_sapp_custmer.json")

df_customer.show()


+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States| AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States| EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States| WDunham@exam

In [13]:
df_customer.printSchema()

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)



In [ ]:
df_customer.columns

In [ ]:
df_customer.describe().show()

cleaning custmer data

In [14]:
df_customer = df_customer.withColumn("SSN",col("SSN").cast(IntegerType()))\
       .withColumn("CUST_ZIP",col("CUST_ZIP").cast(IntegerType()))\
       .withColumn("LAST_UPDATED",col("LAST_UPDATED").cast(TimestampType()))\
       .withColumn("CUST_PHONE",col("CUST_PHONE").cast(StringType())) 
df_customer = df_customer.withColumn("FIRST_NAME", initcap(col('FIRST_NAME'))).withColumn("LAST_NAME",initcap(col('LAST_NAME')))
df_customer = df_customer.withColumn('MIDDLE_NAME',lower(col('MIDDLE_NAME')))
df_customer = df_customer.withColumn("FULL_STREET_ADDRESS",concat_ws(",",col('APT_NO'),col('STREET_NAME'))).drop("APT_NO").drop("STREET_NAME")
df_customer = df_customer.withColumn("CUST_PHONE", regexp_replace(df_customer.CUST_PHONE, "(\d{3})(\d{3})(\d{1})", "($1) $2-$3"))

In [ ]:
df_customer.printSchema()

In [ ]:
df_customer.show()

In [15]:
df_customer.write.format("jdbc").mode("append")\
        .option("url","jdbc:mysql://localhost:3306/creditcard_capstone")\
        .option("dbtable","creditcard_capstone.CDW_SAPP_CUSTOMER")\
        .option("user","root")\
        .option("password","password")\
        .save()

Changing the datatypes

Convert the first_name to Title case and lastname

change the middle name to lowercase

Concatenate aptno and streetname with comma and give column name full_street_address